# Data Clean Up

Libraries

In [27]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import datetime
import numpy as np
import xlwings as xw
from pathlib import Path
import fred_functions as ff
import index_prices as ip
import functions as fp

### ETF Daily Returns

Import ETF file

In [28]:
df_prices = pd.read_excel('../df_prices.xlsx',index_col=0,sheet_name='Sheet1')

The index funds "SPY" and "VBMFX" will be used for this project. 

SPY is an exchange-traded fund that tracks the performance of the S&P 500.
SPY inception was in 1993 while 
VBMFX is a mutual fund that tracks the eprformance of the Bloomberg U.S. Aggregate Float Adjusted Index.
VBMFX was started in 1986.

In [29]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9919 entries, 1986-05-19 to 2025-09-30
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ACWX    4404 non-null   float64
 1   AGG     5537 non-null   float64
 2   EFA     6059 non-null   float64
 3   EWC     7433 non-null   float64
 4   EWG     7433 non-null   float64
 5   EWH     7433 non-null   float64
 6   EWJ     7433 non-null   float64
 7   EWS     7433 non-null   float64
 8   EWU     7433 non-null   float64
 9   IEF     5831 non-null   float64
 10  MDY     7653 non-null   float64
 11  SHY     5831 non-null   float64
 12  SPY     8224 non-null   float64
 13  TLT     5831 non-null   float64
 14  VBMFX   9775 non-null   float64
 15  VFISX   8542 non-null   float64
 16  VFITX   8542 non-null   float64
 17  VGTSX   7404 non-null   float64
 18  VT      4343 non-null   float64
 19  VTIAX   3732 non-null   float64
 20  VTMGX   6571 non-null   float64
 21  VUSTX   9919 non-nu

### Stock Price Data Preprocessing

Transformations Applied:

1. Resampled to take the month-end stock prices
2. Calculate month-over-month change
3. Calculate the rolling 3-month average return and rolling 3-month standard deviation
4. Annualize the rolling 3-month average and std 

In [30]:
df_prices = df_prices[['SPY','VBMFX']] # select tickets
# df = df.loc['1993-02-01':'2025-09-30'] # filter for dates
df_prices.index = pd.to_datetime(df_prices.index)
df_prices = df_prices.sort_index() # sorting
df_prices = df_prices.resample("ME").last().dropna(how='all') # get the month end data, dropnas
df_simple_returns = df_prices.pct_change().dropna(how='all') # find the percent change from the prev month, dropnas
df_cumm_returns = ip.cummulative_return(df_simple_returns,['SPY','VBMFX']) 
df_ln_returns = ip.ln_return(df_simple_returns,['SPY','VBMFX'])
df_funds = df_cumm_returns.combine_first(df_ln_returns).sort_index()
df_funds = df_funds.rename(columns={'SPY':"SPY_mom",'VBMFX':"VBMFX_mom"})
df_funds = df_funds.combine_first(df_prices).sort_index()


In [31]:
df_funds.loc['1993-04-01':'2025-09-30'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 390 entries, 1993-04-30 to 2025-09-30
Freq: ME
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SPY              390 non-null    float64
 1   SPY_3m           390 non-null    float64
 2   SPY_3m_ann       390 non-null    float64
 3   SPY_3m_ann_sd    390 non-null    float64
 4   SPY_3m_sd        390 non-null    float64
 5   SPY_ann_ln_3m    390 non-null    float64
 6   SPY_ln_3m        390 non-null    float64
 7   SPY_mom          390 non-null    float64
 8   VBMFX            390 non-null    float64
 9   VBMFX_3m         390 non-null    float64
 10  VBMFX_3m_ann     390 non-null    float64
 11  VBMFX_3m_ann_sd  390 non-null    float64
 12  VBMFX_3m_sd      390 non-null    float64
 13  VBMFX_ann_ln_3m  390 non-null    float64
 14  VBMFX_ln_3m      390 non-null    float64
 15  VBMFX_mom        390 non-null    float64
dtypes: float64(16)
memory usage: 51.8 

## Macro Indicators (FRED)

import macro File

In [32]:
df_macro = pd.read_excel('../df_macro.xlsx',index_col=0,sheet_name='Sheet1')
# df_macro = df_macro.loc['1990-01-01':'2025-09-30'] # filter for dates

In [33]:
df_macro.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14884 entries, 1985-01-01 to 2025-10-01
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RealGDP           162 non-null    float64
 1   UnemploymentRate  488 non-null    float64
 2   Permits           488 non-null    float64
 3   CPI               489 non-null    float64
 4   CoreCPI           489 non-null    float64
 5   IndustrialProd    488 non-null    float64
 6   RetailTrade       403 non-null    float64
 7   Mort30Y           2126 non-null   float64
 8   UST10Y            10192 non-null  float64
 9   UST2Y             10192 non-null  float64
 10  UST3M             10192 non-null  float64
 11  CorpBAA           9975 non-null   float64
 12  CorpAAA           10228 non-null  float64
 13  CorpTotRetIndex   10132 non-null  float64
 14  FedFundsRate      14884 non-null  float64
dtypes: float64(15)
memory usage: 1.8 MB


### GDP Data Preprocessing

#### GDP
To prepare quarterly GDP data for our model, we perform a two-step process to handle GDP data frequency and reporting lag:

GDP is reported on a quarterly basis. For our research we are using Real Gross Domestic Product (GDPC1) from the FRED website

Units: Billions of Chained 2017 Dollars

**Source:** [FRED GDPC1](https://fred.stlouisfed.org/series/GDPC1)

Transformations Applied:

1.  Since GDP is reported quarterly, we will use the foward-fill method to fill-in the blank months.The GDP value for a quarter is the same for all the months within that quarter. For example, the Q1 GDP value is used for January, February, and March.

2.  Simulate Reporting Lag: In practice, GDP figures are not known in real-time (January's final GDP might not be available until April). To simulate this delay and prevent lookahead bias, we shift the resampled monthly data forward by 3 months.

3. Calculate Year-over-year and month-over-month changes

In [34]:
df_gdp = ff.forward_fill(df_macro[['RealGDP']].copy()) #forward fill to fill in blanks
df_gdp = ff.resample_me(df_gdp) # resampled for month-end value
df_gdp_lag = ff.shift_lag(df_gdp,lag_months=3).rename(columns={'RealGDP':'RealGDP_lag'}) # shift the values by 3 months
df_gdp_yoy = ff.yoy_change(df_gdp_lag).rename(columns={'RealGDP_lag':'RealGDP_yoy'}) # calc yoy change and create df
df_gdp_mom = ff.mom_change(df_gdp_lag).rename(columns={'RealGDP_lag':'RealGDP_mom'}) # calc mom change and create df
df_gdp_clean = df_gdp.join([df_gdp_lag,df_gdp_yoy,df_gdp_mom], how='inner') # only using lag3, combine dfs

In [35]:
df_gdp_clean.tail(5)

,RealGDP,RealGDP_lag,RealGDP_yoy,RealGDP_mom
DATE,,,,
2025-06-30,23770.976,23548.210,0.020193,0.00000
2025-07-31,23770.976,23770.976,0.020805,0.00946
2025-08-31,23770.976,23770.976,0.020805,0.00000
2025-09-30,23770.976,23770.976,0.020805,0.00000
2025-10-31,23770.976,23770.976,0.012454,0.00000


### Permits, CPI, CoreCPI, IndustrialProd, and RetailTrade Data Preprocessing

These key economic indicators are grouped together in the data preprocessing because they are reported monthly.

Transformations Applied:

1.  The macroeconomic indicators mentioned above are reported on a monthly basis, we will use the foward-fill method to fill-in the blank months

2.  Simulate Reporting Lag: In practice, these indicator are not known in real-time (January's final CPI might not be available until February). To simulate this delay and prevent lookahead bias, we shift the resampled monthly data forward by 1 month. This lag is appied to Permits,CPI, CoreCPI, IndustrialProd, and RealRetail.

3. Calculate YoY, MoM, and 3-month rolling average


In [36]:
indicator_index_cols = ['Permits','CPI','CoreCPI','IndustrialProd','RetailTrade'] # make a list of column headers to update
df_indicator_index = df_macro[indicator_index_cols].copy() # make a copy
df_indicator_index = ff.forward_fill(df_indicator_index) # forward fill
df_indicator_index = ff.resample_me(df_indicator_index) # get month-end values
df_lag1_indicator_index = ff.shift_lag(df_indicator_index,lag_months=1) #shift by 1 to account for information lag

In [37]:
# dictionary to rename original numbers to "_lag1"
col_rename_indicator_index = {
    'Permits':'Permits_Lag',
    'CPI':'CPI_Lag',
    'CoreCPI':'CoreCPI_Lag',
    'IndustrialProd':'IndustrialProd_Lag',
    'RetailTrade':'RetailTrade_Lag'
}
col_rename_yoy = {
    'Permits_Lag':'PermitsLag_yoy',
    'CPI_Lag':'CPILag_yoy',
    'CoreCPI_Lag':'CoreCPILag_yoy',
    'IndustrialProd_Lag':'IndustrialProdLag_yoy',
    'RetailTrade_Lag':'RetailTrade_Lag_yoy'
}
ff.rename_columns(df_lag1_indicator_index,col_rename_indicator_index) #ff to rename columns using the dict above
df_indicator_yoy = ff.yoy_change(df_lag1_indicator_index) # ff for year over year change
ff.rename_columns(df_indicator_yoy,col_rename_yoy) # rename yoy columns
df_indicator_mom = ff.mom_change(df_lag1_indicator_index) ## ff for month-over-month change
df_indicator_mom = ff.rolling_3m(df_indicator_mom,df_indicator_mom.columns) # ff to calculate 3month rolling avg, and also annualized 3month rolling avg
df_indicator_clean = df_indicator_index.join([df_indicator_yoy,df_indicator_mom], how='inner')

In [38]:
df_indicator_clean.tail(5)

,Permits,CPI,CoreCPI,IndustrialProd,RetailTrade,PermitsLag_yoy,CPILag_yoy,CoreCPILag_yoy,IndustrialProdLag_yoy,RetailTrade_Lag_yoy,Permits_Lag_r3m,Permits_Lag_ann_r3m,CPI_Lag_r3m,CPI_Lag_ann_r3m,CoreCPI_Lag_r3m,CoreCPI_Lag_ann_r3m,IndustrialProd_Lag_r3m,IndustrialProd_Lag_ann_r3m,RetailTrade_Lag_r3m,RetailTrade_Lag_ann_r3m
DATE,,,,,,,,,,,,,,,,,,,,
2025-06-30,1393.0,321.500,327.600,104.2115,624146.0,-0.009240,0.023759,0.027671,0.006577,0.029423,-0.013653,-0.152077,0.000839,0.010115,0.001411,0.017059,-0.000684,-0.008178,0.000759,0.009147
2025-07-31,1362.0,322.132,328.656,103.8194,628620.0,-0.046543,0.026727,0.029079,0.009279,0.040280,-0.020082,-0.216070,0.001962,0.023805,0.001983,0.024056,0.002157,0.026195,-0.000490,-0.005865
2025-08-31,1330.0,323.364,329.793,103.9203,628620.0,-0.051532,0.027318,0.030486,0.012683,0.037645,-0.014221,-0.157914,0.001881,0.022812,0.002268,0.027561,0.000640,0.007710,0.002753,0.033535
2025-09-30,1330.0,324.368,330.542,103.9203,628620.0,-0.098916,0.029392,0.031122,0.008743,0.041246,-0.015489,-0.170822,0.002887,0.035196,0.002988,0.036457,0.000853,0.010283,0.005684,0.070384
2025-10-31,1330.0,324.368,330.542,103.9203,628620.0,-0.072524,0.030227,0.030255,0.012914,0.032724,-0.015250,-0.168402,0.002965,0.036167,0.002985,0.036410,-0.000930,-0.011106,0.002389,0.029053


#### Permits

Permits is the number of housing permits of new privately-owned housing units. It is an annualized estimate based on that month's  seasonally adjusted pace.

Units: Thousands of Units, SAAS
[Building Permit Survey link](https://www.census.gov/construction/bps/about.html)

**Source:** [FRED PERMIT](https://fred.stlouisfed.org/series/PERMIT)

In [39]:
df_indicator_clean.filter(like='Permits', axis=1).tail(5)

,Permits,PermitsLag_yoy,Permits_Lag_r3m,Permits_Lag_ann_r3m
DATE,,,,
2025-06-30,1393.0,-0.009240,-0.013653,-0.152077
2025-07-31,1362.0,-0.046543,-0.020082,-0.216070
2025-08-31,1330.0,-0.051532,-0.014221,-0.157914
2025-09-30,1330.0,-0.098916,-0.015489,-0.170822
2025-10-31,1330.0,-0.072524,-0.015250,-0.168402


#### CPI (Consumer Price Index) and CoreCPI

CPI measures the average change over time in the prices paid by consumers.
CPI CORE is CPU but excludes the prices of food and energy.

Units: Index 1982-1984=100

**Source:** [FRED CPIAUCSL](https://fred.stlouisfed.org/series/CPIAUCSL)\
**Source:** [FRED CPILFESL](https://fred.stlouisfed.org/series/CPILFESL)

In [40]:
df_indicator_clean.filter(like='CPI', axis=1).tail(6)

,CPI,CoreCPI,CPILag_yoy,CoreCPILag_yoy,CPI_Lag_r3m,CPI_Lag_ann_r3m,CoreCPI_Lag_r3m,CoreCPI_Lag_ann_r3m
DATE,,,,,,,,
2025-05-31,320.580,326.854,0.023337,0.027815,0.001289,0.015582,0.001733,0.020996
2025-06-30,321.500,327.600,0.023759,0.027671,0.000839,0.010115,0.001411,0.017059
2025-07-31,322.132,328.656,0.026727,0.029079,0.001962,0.023805,0.001983,0.024056
2025-08-31,323.364,329.793,0.027318,0.030486,0.001881,0.022812,0.002268,0.027561
2025-09-30,324.368,330.542,0.029392,0.031122,0.002887,0.035196,0.002988,0.036457
2025-10-31,324.368,330.542,0.030227,0.030255,0.002965,0.036167,0.002985,0.036410


#### Industrial Production (IP)

The industrial production (IP) index measures the real output of the manufacturing, mining, and electric and gas utilities industries;

Units: Index 2017=100,Seasonally Adjusted

**Source:** [FRED INDPRO](https://fred.stlouisfed.org/series/INDPRO/)

In [41]:
df_indicator_clean.filter(like='IndustrialProd', axis=1).tail(6)

,IndustrialProd,IndustrialProdLag_yoy,IndustrialProd_Lag_r3m,IndustrialProd_Lag_ann_r3m
DATE,,,,
2025-05-31,103.6570,0.012365,0.002412,0.029334
2025-06-30,104.2115,0.006577,-0.000684,-0.008178
2025-07-31,103.8194,0.009279,0.002157,0.026195
2025-08-31,103.9203,0.012683,0.000640,0.007710
2025-09-30,103.9203,0.008743,0.000853,0.010283
2025-10-31,103.9203,0.012914,-0.000930,-0.011106


#### Retail Sales: Retail Trade

The retail sales measures the total revenue generated by retail stores from the sale or goods and services.

Units: Millions of Dollars, Seasonally Adjusted

**Source:** [FRED RetailTrade](https://fred.stlouisfed.org/series/MRTSSM44000USS)

In [42]:
df_indicator_clean.filter(like='RetailTrade', axis=1).tail(6)

,RetailTrade,RetailTrade_Lag_yoy,RetailTrade_Lag_r3m,RetailTrade_Lag_ann_r3m
DATE,,,,
2025-05-31,618037.0,0.046745,0.004092,0.050222
2025-06-30,624146.0,0.029423,0.000759,0.009147
2025-07-31,628620.0,0.040280,-0.000490,-0.005865
2025-08-31,628620.0,0.037645,0.002753,0.033535
2025-09-30,628620.0,0.041246,0.005684,0.070384
2025-10-31,628620.0,0.032724,0.002389,0.029053


### Unemployment Data Preprocessing

#### Unemployment

These key economic indicators are grouped together in the data preprocessing because they are reported monthly.

Units: Percent, Seasonally Adjusted

Transformations Applied:

1.  The unemployment rate is reported monthly. Any missing months will be filled using forward-fill.

2. To simulate the release delay and avoid lookahead bias, the data is shifted forward by one month.

3. Th unemployment rate, in percent, is converted in decimal form to be consistent with the other features.

4. Calculate the YoY, MoM, and 3-month rolling avg

In [43]:
df_unemployment = df_macro[["UnemploymentRate"]].copy()
df_unemployment = df_macro[["UnemploymentRate"]] /100
df_unemployment = ff.forward_fill(df_unemployment)
df_unemployment = ff.resample_me(df_unemployment)
df_unemployment_lag = ff.shift_lag(df_unemployment,lag_months=1)


df_unemployment_mom = ff.mom_change(df_unemployment_lag)
col_rename_mom = {
    'UnemploymentRate':'UnRate_mom'}
ff.rename_columns(df_unemployment_mom,col_rename_mom) #ff to rename columns using the dict above

df_unemployment_3m = df_unemployment_lag[["UnemploymentRate"]].rolling(window=3).mean()
col_rename_3m = {
    'UnemploymentRate':'UnRate_3m'}
ff.rename_columns(df_unemployment_3m,col_rename_3m) #ff to rename columns using the dict above

df_unemployment_yoy = ff.yoy_change(df_unemployment_lag)
col_rename_yoy = {
    'UnemploymentRate':'UnRate_yoy'}
ff.rename_columns(df_unemployment_yoy,col_rename_yoy) #ff to rename columns using the dict above
df_unemployment_clean = df_unemployment.join([df_unemployment_mom,df_unemployment_3m,df_unemployment_yoy], how='inner') 



In [44]:
df_unemployment_clean

,UnemploymentRate,UnRate_mom,UnRate_3m,UnRate_yoy
DATE,,,,
1985-01-31,0.073,NaN,NaN,NaN
1985-02-28,0.072,NaN,NaN,NaN
1985-03-31,0.072,-0.013699,NaN,NaN
1985-04-30,0.073,0.000000,0.072333,NaN
1985-05-31,0.072,0.013889,0.072333,NaN
...,...,...,...,...
2025-06-30,0.041,0.000000,0.042000,0.05000
2025-07-31,0.042,-0.023810,0.041667,0.00000
2025-08-31,0.043,0.024390,0.041667,0.00000


### 

### Rates - Mortgage 30 Year, US 10 YR Treasury, US 2 Year Treasury, US 3 Month Treasury, TIPS10 YR, CorpBAA, CorpAAA, FedFundsRate

In [45]:
df_rate_clean = ff.credit_spread(df_macro,lag_months=0,delta_calc=True)
# col_remove = ['Mort30Y', 'UST10Y', 'UST2Y', 'UST3M', 'CorpBAA', 'CorpAAA','FedFundsRate']
# df_rate_clean.drop(columns=col_remove,inplace=True)

### Combine all macro features

In [46]:
df_macro_clean = (
    df_gdp_clean.combine_first(df_indicator_clean)
       .combine_first(df_unemployment_clean)
       .combine_first(df_rate_clean)
       .sort_index()
)

### Combine index fund df and macro df

In [47]:
df_clean =(
    df_funds.combine_first(df_macro_clean)
    .sort_index()
)

In [48]:
df_clean.shape

(490, 83)

### Rearrange column

In [49]:
column_order = fp.column_order()
print(len(column_order))
df_clean = df_clean[column_order]

90


### Filter data

In [50]:
df_clean = df_clean.loc['1993-04-01':'2025-09-30'] # filter for dates

In [51]:
df_clean

,SPY,SPY_mom,SPY_3m,SPY_3m_ann,SPY_3m_sd,SPY_3m_ann_sd,SPY_ln_3m,SPY_ann_ln_3m,VBMFX,VBMFX_mom,...,Slope_10Y_FF_Delta3m,AAA_minus_10Y,AAA_minus_10Y_Delta1m,AAA_minus_10Y_Delta3m,BAA_minus_10Y,BAA_minus_10Y_Delta1m,BAA_minus_10Y_Delta3m,BAA_minus_AAA,BAA_minus_AAA_Delta1m,BAA_minus_AAA_Delta3m
1993-04-30,24.480000,-0.025587,0.006867,0.027754,0.025016,0.050031,-0.025920,0.027376,2.382549,0.007438,...,-0.33,1.36,-0.24,-0.05,2.12,-0.06,0.01,0.76,0.18,0.06
1993-05-31,25.140213,0.026970,0.023107,0.095680,0.029114,0.058228,0.026612,0.091375,2.385895,0.001405,...,0.29,1.30,-0.06,-0.28,2.10,-0.02,-0.10,0.80,0.04,0.18
1993-06-30,25.230892,0.003607,0.004302,0.017319,0.026332,0.052664,0.003600,0.017170,2.428994,0.018064,...,-0.32,1.44,0.14,-0.16,2.16,0.06,-0.02,0.72,-0.08,0.14
1993-07-31,25.108412,-0.004854,0.025670,0.106704,0.016483,0.032966,-0.004866,0.101386,2.443795,0.006094,...,-0.28,1.25,-0.19,-0.11,2.05,-0.11,-0.07,0.80,0.08,0.04
1993-08-31,26.070768,0.038328,0.037015,0.156484,0.022883,0.045767,0.037612,0.145384,2.485172,0.016931,...,-0.88,1.20,-0.05,-0.10,1.91,-0.14,-0.19,0.71,-0.09,-0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,586.028015,0.062845,-0.005078,-0.020156,0.059701,0.119402,0.060949,-0.020362,9.469428,-0.007126,...,0.17,1.10,-0.11,0.13,1.84,-0.12,0.27,0.74,-0.01,0.14
2025-06-30,616.141785,0.051386,0.107772,0.505918,0.038411,0.076822,0.050110,0.409403,9.618223,0.015713,...,0.01,1.10,0.00,0.02,1.75,-0.09,-0.01,0.65,-0.09,-0.03
2025-07-31,630.332458,0.023032,0.143197,0.707987,0.020495,0.040991,0.022770,0.535316,9.589507,-0.002986,...,0.20,1.04,-0.06,-0.17,1.67,-0.08,-0.29,0.63,-0.02,-0.12
2025-08-31,643.266602,0.020520,0.097672,0.451746,0.017142,0.034284,0.020312,0.372767,9.669253,0.008316,...,-0.18,1.19,0.15,0.09,1.80,0.13,-0.04,0.61,-0.02,-0.13


### Export data to excel

In [52]:
def to_excel(df,file_name):
    out = Path.home() /"Documents"/"cs668"/"CS668_Capstone"/file_name
    df.to_excel(out, index=True)  # index=True is default; keep if you want the index

# Call to_excel function to save as excel
to_excel(df_clean,"df_SPY_VBMFX_Macro_Signals.xlsx")
